# Lesson 14: 빅데이터와 시각화 – 크롤링 + 시각화

최규빈  
2024-01-10

<a href="https://colab.research.google.com/github/guebin/PP2024WIN/blob/main/posts/Day3/ls14.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

In [1]:
# !pip install yfinance
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import yfinance as yf

# `yfinance`를 이용한 주식자료 시각화

## A. 크롤링 + 데이터정리

`-` yahoo finance: https://finance.yahoo.com/

**Apple: `'AAPL'`**

<img
src="ls14_files/figure-ipynb/8f6bf0b8-35e6-4324-ae0a-e2628e7bb60b-1-10wk-1_files/figure-ipynb/c2016753-3330-437a-bd4e-d9ef1cd24afb-1-825f3296-8485-439d-8432-e52a19c50910.png"
alt="Apple 코드" />

Apple 코드

**삼성전자: `'005930.KS'`**

<img
src="ls14_files/figure-ipynb/8f6bf0b8-35e6-4324-ae0a-e2628e7bb60b-2-10wk-1_files/figure-ipynb/dd41452f-2b17-47e5-8b5d-ea2c46bba973-1-c2ae28ce-13f7-44e7-8c1f-32055d0535f1.png"
alt="삼성전자 코드" />

삼성전자 코드

`-` 크롤링을 위한 코드

In [2]:
symbols = ['AMZN','AAPL','GOOG','MSFT','NFLX','NVDA','TSLA']
start = '2020-01-01'
end = '2024-01-10'
df = yf.download(symbols,start,end)
df

[                       0%%                      ][                       0%%                      ][********************* 43%%                      ]  3 of 7 completed[**********************57%%*                     ]  4 of 7 completed[**********************71%%********              ]  5 of 7 completed[**********************86%%***************       ]  6 of 7 completed[*********************100%%**********************]  7 of 7 completed

## B. 시각화

In [3]:
df.loc[:,'Adj Close'].plot.line(backend='plotly')

/home/cgb2/anaconda3/envs/r/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


# 출산율 시각화

## A. 크롤링 + 데이터정리

`-` 대한민국의 저출산문제

ref: <https://ko.wikipedia.org/wiki/대한민국의_저출산>

`-` 위의 url에서 5번째 테이블을 읽고싶다.

-   5번째 테이블: 시도별 출생아 수

In [4]:
df_lst = pd.read_html('https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%A0%80%EC%B6%9C%EC%82%B0')
df = df_lst[4]
df

## B. 시각화1: 전국 출생아수 시각화

In [5]:
df.set_index('지역/연도[6]')\
.applymap(lambda x: 0 if x == '-' else float(x))\
.iloc[:-1,:]\
.sum(axis=0)\
.plot.line(backend='plotly')

/tmp/ipykernel_1314116/925425684.py:2: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


## C. 시각화2: 시도별 출생아수 시각화 (line)

In [6]:
df.set_index('지역/연도[6]')\
.applymap(lambda x: 0 if x == '-' else float(x)).T\
.loc[:,'서울':'제주']\
.plot.line(backend='plotly')

/tmp/ipykernel_1314116/2975889792.py:2: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


## D. 시각화3: 시도별 출생아수 시각화 (area)

`-` 시각화1,시각화2의 정보가 적절히 혼합되어있는 시각화는 없을까?

In [7]:
df.set_index('지역/연도[6]')\
.applymap(lambda x: 0 if x == '-' else float(x)).T\
.loc[:,'서울':'제주']\
.plot.area(backend='plotly')

/tmp/ipykernel_1314116/1346429013.py:2: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


## E. 시각화1,2,3 수정

`-` 시각화1의 수정

In [8]:
df.set_index('지역/연도[6]')\
.applymap(lambda x: 0 if x == '-' else float(x))\
.applymap(lambda x: x*1000 if x<1000 else x)\
.iloc[:-1,:]\
.sum(axis=0)\
.plot.line(backend='plotly')

/tmp/ipykernel_1314116/1644146122.py:2: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/tmp/ipykernel_1314116/1644146122.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [9]:
df.set_index('지역/연도[6]')\
.applymap(lambda x: 0 if x == '-' else float(x))\
.applymap(lambda x: x*1000 if x<1000 else x).T\
.loc[:,'서울':'제주']\
.plot.line(backend='plotly')

/tmp/ipykernel_1314116/2864061887.py:2: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/tmp/ipykernel_1314116/2864061887.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.


In [10]:
df.set_index('지역/연도[6]')\
.applymap(lambda x: 0 if x == '-' else float(x))\
.applymap(lambda x: x*1000 if x<1000 else x).T\
.loc[:,'서울':'제주']\
.plot.area(backend='plotly')

/tmp/ipykernel_1314116/869707808.py:2: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

/tmp/ipykernel_1314116/869707808.py:3: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.
